In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=244e88e858e361facebcf26d14386fc0a80bee547dd1d5058db322c4df1b1a3a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [3]:
!pip install torchviz graphviz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [4]:
import torch
import datasets
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import get_cosine_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
from rouge_score import rouge_scorer
import os
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]+validation[:1%]")  # Small subset for demo
train_data = dataset.select(range(0, int(0.8 * len(dataset))))
val_data = dataset.select(range(int(0.8 * len(dataset)), len(dataset)))

# Tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Custom Dataset
class CNNDailyMailDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_input_len=512, max_target_len=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        article = self.dataset[idx]["article"]
        summary = self.dataset[idx]["highlights"]

        source = self.tokenizer(
            article,
            max_length=self.max_input_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target = self.tokenizer(
            summary,
            max_length=self.max_target_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": source["input_ids"].squeeze(0),
            "attention_mask": source["attention_mask"].squeeze(0),
            "labels": target["input_ids"].squeeze(0)
        }

# Dataloaders
train_dataset = CNNDailyMailDataset(train_data, tokenizer)
val_dataset = CNNDailyMailDataset(val_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

# Model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
model = model.to(device)

# Training configuration
EPOCHS = 20
STEPS_PER_EPOCH = min(500, len(train_loader))
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 3e-5
WARMUP_STEPS = int(0.1 * EPOCHS * STEPS_PER_EPOCH)
TOTAL_TRAINING_STEPS = EPOCHS * STEPS_PER_EPOCH

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=TOTAL_TRAINING_STEPS
)
scaler = GradScaler()

# ROUGE evaluation
def evaluate(model, val_loader, tokenizer):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougel = 0, 0, 0
    count = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating", leave=False):
            batch = {k: v.to(device) for k, v in batch.items()}
            generated_ids = model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            targets = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

            for pred, target in zip(preds, targets):
                scores = scorer.score(target, pred)
                rouge1 += scores["rouge1"].fmeasure
                rouge2 += scores["rouge2"].fmeasure
                rougel += scores["rougeL"].fmeasure
                count += 1

    return {
        "rouge1": rouge1 / count,
        "rouge2": rouge2 / count,
        "rougeL": rougel / count
    }

# Early stopping
best_loss = float("inf")
patience = 3
patience_counter = 0

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()
    loop = tqdm(train_loader, total=STEPS_PER_EPOCH, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)

    for step, batch in enumerate(loop):
        if step >= STEPS_PER_EPOCH:
            break
        batch = {k: v.to(device) for k, v in batch.items()}

        with autocast():
            outputs = model(**batch)
            loss = outputs.loss
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        loop.set_postfix(loss=loss.item() * GRADIENT_ACCUMULATION_STEPS)

    avg_train_loss = total_loss / STEPS_PER_EPOCH
    print(f"\n✅ Epoch {epoch+1} completed | Avg Loss: {avg_train_loss:.4f}")

    # Validation
    val_scores = evaluate(model, val_loader, tokenizer)
    print(f"📊 Validation ROUGE Scores - R1: {val_scores['rouge1']:.4f}, R2: {val_scores['rouge2']:.4f}, RL: {val_scores['rougeL']:.4f}")

    # Early stopping
    if avg_train_loss < best_loss:
        best_loss = avg_train_loss
        model.save_pretrained("best_bart_model")
        tokenizer.save_pretrained("best_bart_model")
        patience_counter = 0
        print("💾 Model improved and saved to 'best_bart_model'")
    else:
        patience_counter += 1
        print(f"⏸️ No improvement. Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break

# Save final model
model.save_pretrained("bart_summarization_model")
tokenizer.save_pretrained("bart_summarization_model")
print("✅ Final model saved to 'bart_summarization_model'")


2025-04-22 06:27:07.883840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745303228.081297      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745303228.138623      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...


✅ Epoch 19 completed | Avg Loss: 0.4936


📊 Validation ROUGE Scores - R1: 0.3499, R2: 0.1385, RL: 0.2437
💾 Model improved and saved to 'best_bart_model'


Epoch 20/20:   0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_31/1721990862.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



✅ Epoch 20 completed | Avg Loss: 0.4595


📊 Validation ROUGE Scores - R1: 0.3566, R2: 0.1383, RL: 0.2460
💾 Model improved and saved to 'best_bart_model'
✅ Final model saved to 'bart_summarization_model'


In [11]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("best_bart_model")
tokenizer = BartTokenizer.from_pretrained("best_bart_model")

model.eval()


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [12]:
def generate_summary(text, model, tokenizer, max_input_length=512, max_output_length=128):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_output_length,
            num_beams=4,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [13]:
text = """The White House on Tuesday said it had nothing to do with a tabloid report claiming President Obama would soon be divorcing First Lady Michelle Obama..."""
summary = generate_summary(text, model, tokenizer)
print("📄 Original Text:\n", text)
print("\n📝 Generated Summary:\n", summary)


📄 Original Text:
 The White House on Tuesday said it had nothing to do with a tabloid report claiming President Obama would soon be divorcing First Lady Michelle Obama...

📝 Generated Summary:
 White House says it had nothing to do with a tabloid report claiming President Obama would soon be divorcing First Lady Michelle Obama .
